In [ ]:
import os
import sys
import random
from joblib import Parallel, delayed
import multiprocessing
import concurrent.futures
import time, random
from numba import jit
print("done")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

%matplotlib inline

# import cv2
from sklearn.model_selection import train_test_split

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add,UpSampling2D
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img#,save_img

In [ ]:
# Set some parameters
img_size_ori = 101
img_size_target = 128
im_width = 101
im_height = 101
im_chan = 1
basicpath = '../input/'
path_train = basicpath + 'train/'
path_test = basicpath + 'test/'

path_train_images = path_train + 'images/'
path_train_masks = path_train + 'masks/'
path_test_images = path_test + 'images/'

def upsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True)
    #res = np.zeros((img_size_target, img_size_target), dtype=img.dtype)
    #res[:img_size_ori, :img_size_ori] = img
    #return res
    
def downsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_ori, img_size_ori), mode='constant', preserve_range=True)
    #return img[:img_size_ori, :img_size_ori]

In [ ]:
# Loading of training/testing ids and depths

train_df = pd.read_csv("../input/train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("../input/depths.csv", index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]
print(len(depths_df))
len(train_df)

In [ ]:
import cv2
def my_load_image(file):
#img = cv2.imread('../input/train/images/9f3b8d0186.png')
    img = cv2.imread(file)
    img_to_yuv = cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
    img_to_yuv[:,:,0] = cv2.equalizeHist(img_to_yuv[:,:,0])
    hist_equalization_result = cv2.cvtColor(img_to_yuv, cv2.COLOR_YUV2BGR)
    #median = cv2.medianBlur(hist_equalization_result,5)
    #kernel = np.ones((5,5),np.float32)/25
    #dst = cv2.filter2D(hist_equalization_result,-1,kernel)
    return hist_equalization_result

train_df["images"] = [np.array(load_img("../input/train/images/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]

In [ ]:
train_df["masks"] = [np.array(load_img("../input/train/masks/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]

In [ ]:
train_df["coverage"] = train_df.masks.map(np.sum) / pow(img_size_ori, 2)

def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
train_df["coverage_class"] = train_df.coverage.map(cov_to_class)


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,5))
sns.distplot(train_df.coverage, kde=False, ax=axs[0])
sns.distplot(train_df.coverage_class, bins=10, kde=False, ax=axs[1])
plt.suptitle("Salt coverage")
axs[0].set_xlabel("Coverage")
axs[1].set_xlabel("Coverage class")

In [ ]:
#Plotting the depth distributions¶

sns.distplot(train_df.z, label="Train")
sns.distplot(test_df.z, label="Test")
plt.legend()
plt.title("Depth distribution")

In [ ]:
# Create train/validation split stratified by salt coverage
ids_train, ids_valid, x_train, x_valid, y_train, y_valid, cov_train, cov_test, depth_train, depth_test = train_test_split(
    train_df.index.values,
    np.array(train_df.images.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1), 
    np.array(train_df.masks.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1), 
    train_df.coverage.values,
    train_df.z.values,
    test_size=0.1, stratify=train_df.coverage_class, random_state=1337)

x_train.shape

In [ ]:


iou_thresholds = np.array([0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])

def fitnes_lossfunction(imgs_true, imgs_pred):
    num_images = len(imgs_true)
    scores = np.zeros(num_images)
    
    for i in range(num_images):
        if imgs_true[i].sum() == imgs_pred[i].sum() == 0:
            scores[i] = 1
        else:
            scores[i] = (iou_thresholds <= iou(imgs_true[i], imgs_pred[i])).mean()
            
    return scores.mean()


In [ ]:
#Data augmentation
x_train = np.append(x_train, [np.fliplr(x) for x in x_train], axis=0)
y_train = np.append(y_train, [np.fliplr(x) for x in y_train], axis=0)
print(x_train.shape)
print(y_valid.shape)

In [ ]:
def small_ae():
    input_img = Input(shape=(128, 128, 1))  # adapt this if using `channels_first` image data format
    net = Conv2D( 32, (5, 5), strides=2, padding='SAME',kernel_initializer='random_uniform')(input_img)
    net = Conv2D(16, (5, 5), strides=2, padding='SAME',kernel_initializer='random_uniform')(net)
    net = Conv2D( 8, (5, 5), strides=4, padding='SAME',kernel_initializer='random_uniform')(net)
    # decoder
    # 2 x 2 x 8    ->  8 x 8 x 16
    # 8 x 8 x 16   ->  16 x 16 x 32
    # 16 x 16 x 32  ->  32 x 32 x 1
    net = Conv2DTranspose( 16, (5, 5), strides=4, padding='SAME',kernel_initializer='random_uniform')(net)
    net = Conv2DTranspose( 32, (5, 5), strides=2, padding='SAME',kernel_initializer='random_uniform')(net)
    decoded  = Conv2DTranspose( 1, (5, 5), strides=2, padding='SAME',kernel_initializer='random_uniform', activation='sigmoid')(net)
    

    autoencoder = Model(input_img, decoded )
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy',metrics=['acc'])
    return autoencoder

def small_unet():
    inputs = Input((128, 128, 1))


    c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (inputs)
    c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
    c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
    c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
    c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (c5)

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
    c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
    c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
    c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    return model
small_unet().summary()

In [ ]:
def model_crossover(model1, model2):
    model=small_ae()
    #for each layer
    w1=model1.get_weights()
    w2=model2.get_weights()
    #print(type(w1))    
    #print(len(w1))
    wf=[]
    for x in range(len(w1)):
        if len(w1[x].shape)==4:
     #       print("4")
            xx=np.random.rand(w1[x].shape[0],w1[x].shape[1],w1[x].shape[2],w1[x].shape[3])    
            mutation_choise=np.random.rand(w1[x].shape[0],w1[x].shape[1],w1[x].shape[2],w1[x].shape[3])    
            mutation_val=np.random.rand(w1[x].shape[0],w1[x].shape[1],w1[x].shape[2],w1[x].shape[3])*2            
            z = np.where(xx>0.5, w1[x], w2[x])
            z = np.where(mutation_choise>0.85,mutation_val*z,z)
            wf.append(z)
        if len(w1[x].shape)==3:
      #      print("3")
            xx=np.random.rand(w1[x].shape[0],w1[x].shape[1],w1[x].shape[2])    
            mutation_choise=np.random.rand(w1[x].shape[0],w1[x].shape[1],w1[x].shape[2])    
            mutation_val=np.random.rand(w1[x].shape[0],w1[x].shape[1],w1[x].shape[2])*2            
            z = np.where(xx>0.5, w1[x], w2[x])
            z = np.where(mutation_choise>0.85,mutation_val*z,z)
            wf.append(z)
        if len(w1[x].shape)==2:
       #     print("2")
            xx=np.random.rand(w1[x].shape[0],w1[x].shape[1])    
            mutation_choise=np.random.rand(w1[x].shape[0],w1[x].shape[1])    
            mutation_val=np.random.rand(w1[x].shape[0],w1[x].shape[1])*2            
            z = np.where(xx>0.5, w1[x], w2[x])
            z = np.where(mutation_choise>0.85,mutation_val*z,z)
            wf.append(z)
        if len(w1[x].shape)==1:
        #    print("1")
            xx=np.random.rand(w1[x].shape[0])    
            mutation_choise=np.random.rand(w1[x].shape[0])    
            mutation_val=np.random.rand(w1[x].shape[0])*2            
            z = np.where(xx>0.5, w1[x], w2[x])
            z = np.where(mutation_choise>0.85,mutation_val*z,z)      
   #         z=np.asscalar(z)
            wf.append(z)
            #print(z.shape)
                                        
    model.set_weights(wf)
 
    return model
      
 
def runtournament():
    list_idx_on_tournament=[]
    for x in range (tournament_sel):
        list_idx_on_tournament.append(int(random.uniform(0, total_models-1)))
        
    best1=-999999999999999
    best2=-999999999999999
    best1_idx=-999999999999999
    best2_idx=-999999999999999
    for  x in range (tournament_sel):
        if fitness[list_idx_on_tournament[x]]>best1:
            best1=fitness[x]
            best1_idx=x
            
    for  x in range (tournament_sel):
        if fitness[list_idx_on_tournament[x]]>best2 and x!=best1_idx:
            best2=fitness[x]
            best2_idx=x        
    return current_pool[list_idx_on_tournament[best1_idx]],current_pool[list_idx_on_tournament[best2_idx]]

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)
def filter_image(img):
    if img.sum() < 100:
        return np.zeros(img.shape)
    else:
        return img

## Scoring for last model
#thresholds = np.linspace(0.3, 0.7, 31)


def parallel_scoring(i,x_train,y_train):    
    
    preds_valid=current_pool[x].predict(x_train)        
    ious = fitnes_lossfunction(y_train.reshape((-1, img_size_target, img_size_target)), [filter_image(img) for img in preds_valid > 0.5 ])    
    
    return ious
    
    
def parallel_muttion(i):
    model1,model2=runtournament()
    model=model_crossover(model1,model2)
    return model




In [ ]:
current_pool = [] #actual networks saved
fitness = [] #save value for each network
total_models = 75 #we load 75 models in random state
generations=50 #we perform and train GA over 50 generationand end
bestfitness_index=0
tournament_sel=5 #usefull for tournament selection method
        
    
# Initialize all models with random weigth
for i in range(total_models):
    # model
    model=small_ae()
    weights = model.get_weights()
    weights = [np.random.permutation(w.flat).reshape(w.shape) for w in weights]
    model.set_weights(weights)
    #model.summary()
    current_pool.append(model)
    fitness.append(-100)    


            


In [ ]:

print("start generations")


#num_cores = multiprocessing.cpu_count()
#inputs = range(total_models)

for i in range(generations):
        print("Generation: ",i)
        results=[]
        #with concurrent.futures.ProcessPoolExecutor() as executor:
            #for out1 in executor.map(parallel_scoring, range(0, total_models)):
                #results.append(out1)
        #xx_train,yy_train=next_batch(128,x_train,y_train)
        for x in range(total_models):
            results.append(parallel_scoring(x,x_train,y_train))                
#        results = Parallel(n_jobs=num_cores)(delayed(parallel_scoring)(current_pool[i]) for i in inputs)
        
        
        fitness=results    
        bestfitness_index = np.argmax(np.array(fitness)) 
        print ('BEst Fitnes val {}- best fitnes index {}',fitness[bestfitness_index],bestfitness_index)
        best_model=current_pool[bestfitness_index]#we save this model and save this as last
        new_pool=[]        
        for x in range(total_models-1):
            model=parallel_muttion(i)
            new_pool.append(model)        
            #with concurrent.futures.ProcessPoolExecutor() as executor:
            #for out1 in executor.map(parallel_muttion, range(0, total_models-1)):
             #   new_pool.append(out1)                                    
        current_pool=new_pool
        current_pool.append(best_model)
        
            

print ('BEst Fitnes val {}- best fitnes index {}',fitness[bestfitness_index],bestfitness_index)
            
            
        